### ⏱ Time Complexity of Package Tracking System Methods

| Function                                | Time Complexity | Description                                                                 |
|-----------------------------------------|-----------------|-----------------------------------------------------------------------------|
| _ShipmentNode.__init__                  | O(1)            | Initializes a shipment status node.                                        |
| _ShipmentHistory.__init__               | O(1)            | Initializes shipment history linked list.                                  |
| _ShipmentHistory.add_status             | O(m)            | Adds a new status at the end of the history list.                          |
| _ShipmentHistory.get_status             | O(m)            | Retrieves all status updates.                                              |
| _PackageNode.__init__                   | O(1)            | Initializes a package node and its first status.                           |
| _DeliveryPriorityQueue.__init__         | O(1)            | Initializes the delivery priority queue.                                   |
| _DeliveryPriorityQueue.add_package      | O(n)            | Inserts package by priority into linked list.                              |
| _DeliveryPriorityQueue.remove_package   | O(n)            | Removes package by ID from the priority queue.                             |
| PackageTrackingSystem.__init__          | O(1)            | Initializes system with queue and dictionary.                              |
| PackageTrackingSystem.add_package       | O(n)            | Adds a new package to queue and dictionary.                                |
| PackageTrackingSystem.update_status     | O(m)            | Adds a new status to package history.                                      |
| PackageTrackingSystem.get_history       | O(m)            | Returns full status history of a package.                                  |
| PackageTrackingSystem.remove_package    | O(n)            | Removes package from queue and dictionary.                                 |
| PackageTrackingSystem.show_delivered_packages | O(n·m)     | Checks latest status of each package to find "Delivered".                  |
| PackageTrackingSystem.view_all_packages | O(n·m)          | Retrieves and displays full status list for all packages.                  |

**Where:**
- `n` → Number of packages in the system  
- `m` → Maximum number of status updates per package


In [ ]:
class _ShipmentNode:
    def __init__(self, status, time, location):
        self.status = status
        self.time = time
        self.location = location
        self.next = None


class _ShipmentHistory:
    def __init__(self):
        self.head = None  

    def add_status(self, status, time, location):
        new_node = _ShipmentNode(status, time, location)
        if not self.head:
            self.head = new_node
        else:
            temp = self.head
            while temp.next:
                temp = temp.next
            temp.next = new_node

    def get_status(self):
        status_list = []
        current = self.head
        while current:
            status_list.append((current.time, current.status, current.location))
            current = current.next
        return status_list


class _PackageNode:
    def __init__(self, package_id, status, time, location, priority):
        self.package_id = package_id
        self.priority = priority
        self.history = _ShipmentHistory()
        self.history.add_status(status, time, location)
        self.next = None


class _DeliveryPriorityQueue:
    def __init__(self):
        self.head = None

    def add_package(self, package_id, status, time, location, priority):
        new_package = _PackageNode(package_id, status, time, location, priority)

        if self.head is None or self.head.priority > priority:
            new_package.next = self.head
            self.head = new_package
        else:
            current = self.head
            while current.next and current.next.priority < priority:
                current = current.next
            while current.next and current.next.priority == priority:
                current = current.next
            new_package.next = current.next
            current.next = new_package

        return new_package

    def remove_package(self, package_id):
        if self.head is None:
            print(f"Cannot remove. Queue is empty.")
            return False

        if self.head.package_id == package_id:
            self.head = self.head.next
            print(f"Package {package_id} removed from queue.")
            return True

        current = self.head
        while current.next and current.next.package_id != package_id:
            current = current.next

        if current.next:
            current.next = current.next.next
            print(f"Package {package_id} removed from queue.")
            return True

        print(f"Package {package_id} not found in queue.")
        return False


class PackageTrackingSystem:
    def __init__(self):
        self.delivery_queue = _DeliveryPriorityQueue()
        self.packages = {}

    def add_package(self, package_id, status, time, location, priority):
        if package_id in self.packages:
            print(f"Error: Package {package_id} already exists!")
            return None
        package = self.delivery_queue.add_package(package_id, status, time, location, priority)
        self.packages[package_id] = package
        return package

    def update_status(self, package_id, new_status, time, location):
        package = self.packages.get(package_id)
        if package:
            package.history.add_status(new_status, time, location)
            print(f"Status for package {package_id} updated to '{new_status}'.")
        else:
            print(f"Error: Package {package_id} not found in the system!")

    def get_history(self, package_id):
        package = self.packages.get(package_id)
        if package:
            return package.history.get_status()
        print(f"Error: Package {package_id} not found in the system!")
        return []

    def remove_package(self, package_id):
        if package_id in self.packages:
            found = self.delivery_queue.remove_package(package_id)
            if found:
                del self.packages[package_id]
                print(f"Package {package_id} successfully removed from the system.")
        else:
            print(f"Package {package_id} not found in the system!")

    def show_delivered_packages(self):
        found = False
        for package_id, package in self.packages.items():
            history = package.history.get_status()
            if history and history[-1][1] == "Delivered":
                print(f"Package ID: {package_id} - Status: Delivered")
                found = True
        if not found:
            print("No packages have been delivered yet.")

    def view_all_packages(self):
        if not self.packages:
            print("No packages in the system.")
            return
        print("\nAll Packages in the System:")
        for package_id, package in self.packages.items():
            history = package.history.get_status()
            if history:
                last_status = history[-1]
                print(f"Package ID: {package_id}, Priority: {package.priority}, Latest Status: {last_status}")


def run_menu():
    system = PackageTrackingSystem()

    # Preloaded test packages
    system.add_package("PKG101", "Dispatched", "2025-04-10 10:00", "Warehouse A", 2)
    system.add_package("PKG102", "In Transit", "2025-04-10 11:00", "City Hub", 3)
    system.add_package("PKG103", "Delivered", "2025-04-09 17:30", "Customer Address", 1)

    while True:
        print("\n----- Package Tracking System Menu -----")
        print("1. Add Package")
        print("2. Update Package Status")
        print("3. Show Delivered Packages")
        print("4. Remove Package")
        print("5. Show Package History")
        print("6. View All Packages")
        print("7. Exit")

        choice = input("Enter your choice (1-7): ")

        if choice == "1":
            package_id = input("Enter Package ID: ")
            status = input("Enter Initial Status: ")
            time = input("Enter Time: ")
            location = input("Enter Location: ")
            priority = int(input("Enter Priority (lower is higher priority): "))
            system.add_package(package_id, status, time, location, priority)

        elif choice == "2":
            package_id = input("Enter Package ID to update: ")
            new_status = input("Enter New Status: ")
            time = input("Enter Time: ")
            location = input("Enter Location: ")
            system.update_status(package_id, new_status, time, location)

        elif choice == "3":
            system.show_delivered_packages()

        elif choice == "4":
            package_id = input("Enter Package ID to remove: ")
            system.remove_package(package_id)

        elif choice == "5":
            package_id = input("Enter Package ID to view history: ")
            history = system.get_history(package_id)
            if history:
                print(f"\nHistory of package {package_id}:")
                for entry in history:
                    print(f"Time: {entry[0]}, Status: {entry[1]}, Location: {entry[2]}")
            else:
                print("No history found.")

        elif choice == "6":
            system.view_all_packages()

        elif choice == "7":
            print("Exiting Package Tracking System.")
            break

        else:
            print("Invalid choice. Please enter a number from 1 to 7.")


run_menu()



----- Package Tracking System Menu -----
1. Add Package
2. Update Package Status
3. Show Delivered Packages
4. Remove Package
5. Show Package History
6. View All Packages
7. Exit


Enter your choice (1-7):  1
Enter Package ID:  a
Enter Initial Status:  wqw
Enter Time:  qe
Enter Location:  qw
Enter Priority (lower is higher priority):  1



----- Package Tracking System Menu -----
1. Add Package
2. Update Package Status
3. Show Delivered Packages
4. Remove Package
5. Show Package History
6. View All Packages
7. Exit


Enter your choice (1-7):  2
Enter Package ID to update:  qwwq
Enter New Status:  qwqw
Enter Time:  qwqw
Enter Location:  qw


Error: Package qwwq not found in the system!

----- Package Tracking System Menu -----
1. Add Package
2. Update Package Status
3. Show Delivered Packages
4. Remove Package
5. Show Package History
6. View All Packages
7. Exit


Enter your choice (1-7):  3


Package ID: PKG103 - Status: Delivered

----- Package Tracking System Menu -----
1. Add Package
2. Update Package Status
3. Show Delivered Packages
4. Remove Package
5. Show Package History
6. View All Packages
7. Exit
